# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from os.path import dirname

#### Creating list of filepaths to process original event csv data files

In [3]:

# get the current folder and subfolder event data
filepath = os.getcwd() + '/../../event_data/'
print(filepath)
print(list(os.walk(filepath)))
# collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

# check number of files
print(len(file_path_list))

/Users/kehindetomiwa/Documents/DS_courses/udacity/data_engineering/data_model/data_modeling_with_cassandra/src/notebook/../../event_data/
[('/Users/kehindetomiwa/Documents/DS_courses/udacity/data_engineering/data_model/data_modeling_with_cassandra/src/notebook/../../event_data/', [], ['2018-11-15-events.csv', '2018-11-22-events.csv', '2018-11-09-events.csv', '2018-11-18-events.csv', '2018-11-04-events.csv', '2018-11-01-events.csv', '2018-11-27-events.csv', '2018-11-10-events.csv', '2018-11-20-events.csv', '2018-11-17-events.csv', '2018-11-06-events.csv', '2018-11-03-events.csv', '2018-11-28-events.csv', '2018-11-12-events.csv', '2018-11-25-events.csv', '2018-11-26-events.csv', '2018-11-11-events.csv', '2018-11-14-events.csv', '2018-11-23-events.csv', '2018-11-08-events.csv', '2018-11-19-events.csv', '2018-11-05-events.csv', '2018-11-02-events.csv', '2018-11-29-events.csv', '2018-11-13-events.csv', '2018-11-24-events.csv', '2018-11-21-events.csv', '2018-11-16-events.csv', '2018-11-07-ev

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    print(f)

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


/Users/kehindetomiwa/Documents/DS_courses/udacity/data_engineering/data_model/data_modeling_with_cassandra/src/notebook/../../event_data/2018-11-15-events.csv
/Users/kehindetomiwa/Documents/DS_courses/udacity/data_engineering/data_model/data_modeling_with_cassandra/src/notebook/../../event_data/2018-11-22-events.csv
/Users/kehindetomiwa/Documents/DS_courses/udacity/data_engineering/data_model/data_modeling_with_cassandra/src/notebook/../../event_data/2018-11-09-events.csv
/Users/kehindetomiwa/Documents/DS_courses/udacity/data_engineering/data_model/data_modeling_with_cassandra/src/notebook/../../event_data/2018-11-18-events.csv
/Users/kehindetomiwa/Documents/DS_courses/udacity/data_engineering/data_model/data_modeling_with_cassandra/src/notebook/../../event_data/2018-11-04-events.csv
/Users/kehindetomiwa/Documents/DS_courses/udacity/data_engineering/data_model/data_modeling_with_cassandra/src/notebook/../../event_data/2018-11-01-events.csv
/Users/kehindetomiwa/Documents/DS_courses/udac

In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster()
    session = cluster.connect()
except Exception as e:
    print(e)

# To establish connection and begin executing queries, need a session


#### Create Keyspace

In [7]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS music_db
        WITH REPLICATION =
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
                    )

except Exception as e:
    print(e)


#### Set Keyspace

In [8]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('music_db')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
df = pd.read_csv('event_datafile_new.csv', encoding='utf8')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Harmonia,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Sehr kosmisch,26
1,The Prodigy,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",583,The Big Gundown,26
2,Train,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Marry Me,26
3,Sony Wonder,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",597,Blackbird,61
4,Van Halen,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",602,Best Of Both Worlds (Remastered Album Version),80


In [10]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS music_history \
            (session_id INT, session_item INT, artist TEXT, song TEXT, length FLOAT, \
            PRIMARY KEY(session_id, session_item))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [11]:
for i, row in df.iterrows():
    query  = "INSERT INTO music_history (session_id, session_item, artist, song, length)"
    query += "VALUES(%s, %s, %s, %s, %s)"
    session.execute(query, (row.sessionId, row.itemInSession, row.artist, row.song, row.length))


#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "SELECT * FROM music_history WHERE session_id=338 AND session_item=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(session_id=338, session_item=4, artist='Faithless', length=495.30731201171875, song='Music Matters (Mark Knight Dub)')


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [13]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query  =  "CREATE TABLE IF NOT EXISTS user_history "
query += "(user_id INT, session_id INT, session_item INT, artist TEXT, song TEXT, user TEXT, \
           PRIMARY KEY((user_id), session_id, session_item))"

try:
    session.execute(query)
except Exception as e:
    print(e)

for i, row in df.iterrows():
    query  = "INSERT INTO user_history (user_id, session_id, session_item, artist, song, user)"
    query += "VALUES(%s, %s, %s, %s, %s, %s)"
    session.execute(query, (row.userId, row.sessionId, row.itemInSession, row.artist, row.song, f'{row.firstName} {row.lastName}'))
                    

query = "SELECT * FROM user_history WHERE user_id=10 AND session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row)

Row(user_id=10, session_id=182, session_item=0, artist='Down To The Bone', song="Keep On Keepin' On", user='Sylvie Cruz')
Row(user_id=10, session_id=182, session_item=1, artist='Three Drives', song='Greece 2000', user='Sylvie Cruz')
Row(user_id=10, session_id=182, session_item=2, artist='Sebastien Tellier', song='Kilometer', user='Sylvie Cruz')
Row(user_id=10, session_id=182, session_item=3, artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user='Sylvie Cruz')


In [14]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query  =  "CREATE TABLE IF NOT EXISTS song_history "
query += "(song TEXT, user TEXT, \
           PRIMARY KEY(song, user))"

try:
    session.execute(query)
except Exception as e:
    print(e)
    
for i, row in df.iterrows():
    query  = "INSERT INTO song_history (song, user)"
    query += "VALUES(%s, %s)"
    session.execute(query, (row.song, f'{row.firstName} {row.lastName}'))
query = "SELECT * FROM song_history WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
data = list(map(lambda row: list(row), rows))
pd.DataFrame(data, columns = ['song', 'user'])    
query = "SELECT * FROM song_history WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)
                    

Row(song='All Hands Against His Own', user='Jacqueline Lynch')
Row(song='All Hands Against His Own', user='Sara Johnson')
Row(song='All Hands Against His Own', user='Tegan Levine')


### Drop the tables before closing out the sessions

In [15]:
## TO-DO: Drop the table before closing out the sessions

In [16]:
session.execute("DROP TABLE IF EXISTS music_history")
session.execute("DROP TABLE IF EXISTS user_history")
session.execute("DROP TABLE IF EXISTS song_history")


### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()